In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df = pd.read_csv("bozuk_veri.csv")

In [3]:
df.head(10)

,Siparis_ID,Tarih,Kategori,Fiyat,Yas,Sehir,Odeme_Yontemi
0,-1,2023-11-24,Kozmetik,443.82 TL,39.0,Ankara,Havale
1,1001,2023-09-16,Spor,$4049.06,33.0,Izmir,Havale
2,1002,2023-12-24,Spor,4239.0,41.0,Antalya,Kredi Kartı
3,1003,2023-07-13,Giyim,238.66,50.0,Ankara,Kredi Kartı
4,1004,2023-07-14,Ev & Yaşam,$3420.9,32.0,ist,Havale
5,1005,2023-10-01,Kitap,1628.17,32.0,Izmir,Kapıda Ödeme
6,1006,2023-07-25,Kozmetik,4424.57,50.0,ist,kredi kartı
7,1007,2023-10-03,Giyim,$463.64,42.0,İstanbul,Havale
8,1008,2023-11-18,Spor,4855.68 TL,30.0,ist,Kredi Kartı
9,1009,2023-12-16,Giyim,4769.54,40.0,Ankara,kredi kartı


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Siparis_ID     5000 non-null   int64  
 1   Tarih          4917 non-null   object 
 2   Kategori       5000 non-null   object 
 3   Fiyat          5000 non-null   object 
 4   Yas            4782 non-null   float64
 5   Sehir          5000 non-null   object 
 6   Odeme_Yontemi  5000 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 273.6+ KB


In [5]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Siparis_ID,5000.0,3489.490000,1447.295057,-1.0,2239.75,3489.5,4739.25,5999.0
Yas,4782.0,35.425136,15.881283,-5.0,28.00,35.0,41.00,150.0


In [6]:
df.isnull().sum()

Siparis_ID         0
Tarih             83
Kategori           0
Fiyat              0
Yas              218
Sehir              0
Odeme_Yontemi      0
dtype: int64

In [7]:
df_clean = df.copy()

In [8]:
df_clean.columns = df_clean.columns.str.lower().str.strip()

In [9]:
df_clean.head()

,siparis_id,tarih,kategori,fiyat,yas,sehir,odeme_yontemi
0,-1,2023-11-24,Kozmetik,443.82 TL,39.0,Ankara,Havale
1,1001,2023-09-16,Spor,$4049.06,33.0,Izmir,Havale
2,1002,2023-12-24,Spor,4239.0,41.0,Antalya,Kredi Kartı
3,1003,2023-07-13,Giyim,238.66,50.0,Ankara,Kredi Kartı
4,1004,2023-07-14,Ev & Yaşam,$3420.9,32.0,ist,Havale


In [10]:
mask = pd.to_numeric(df_clean["fiyat"], errors="coerce").isna()

# Sadece price sütununu göster
non_numeric_prices = df_clean.loc[mask, "fiyat"]
print(non_numeric_prices)

0        443.82 TL
1         $4049.06
4          $3420.9
7          $463.64
8       4855.68 TL
           ...    
4959       $1358.0
4961       $174.66
4972       $981.52
4986      $1824.25
4991       $901.76
Name: fiyat, Length: 748, dtype: object


In [11]:
# replace regex=True diyerek sayı ve nokta dışındaki her şeyi silebiliriz ama
# daha kontrollü gitmek için manuel siliyorum:
df_clean['fiyat'] = df_clean['fiyat'].astype(str).str.replace('$', '', regex=False)
df_clean['fiyat'] = df_clean['fiyat'].str.replace(' TL', '', regex=False)

In [12]:
# pd.to_numeric kullanıyoruz. 'coerce' parametresi çok önemlidir. 
# Eğer veri sayıya dönüşemiyorsa (örn: "Hatalı"), program çökmez, oraya NaN (boş) yazar.
df_clean['fiyat'] = pd.to_numeric(df_clean['fiyat'], errors='coerce')

In [13]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   siparis_id     5000 non-null   int64  
 1   tarih          4917 non-null   object 
 2   kategori       5000 non-null   object 
 3   fiyat          5000 non-null   float64
 4   yas            4782 non-null   float64
 5   sehir          5000 non-null   object 
 6   odeme_yontemi  5000 non-null   object 
dtypes: float64(2), int64(1), object(4)
memory usage: 273.6+ KB


In [14]:
df_clean['tarih'] = pd.to_datetime(df_clean['tarih'], errors='coerce')

In [15]:
df_clean["sehir"].value_counts()

sehir
ist         760
Bursa       716
Ankara      713
İstanbul    710
Izmir       708
Antalya     702
İzmir       691
Name: count, dtype: int64

In [16]:
city_corrections = {
    'ist': 'İstanbul',
    'izmir': 'İzmir',
    "Izmir": "İzmir"
}

In [17]:
df_clean['sehir'] = df_clean['sehir'].str.strip()

In [18]:
df_clean['sehir'] = df_clean['sehir'].replace(city_corrections)

In [19]:
df_clean["odeme_yontemi"].value_counts()

odeme_yontemi
Kapıda Ödeme    1033
Kredi  Kartı    1025
kredi kartı      985
Kredi Kartı      982
Havale           975
Name: count, dtype: int64

In [20]:
df_clean['odeme_yontemi'] = (
    df_clean['odeme_yontemi']
    .str.replace(r'\s+', ' ', regex=True)  # birden fazla boşluğu tek boşluk yap
    .str.strip()                            # baş ve sondaki boşlukları temizle
)

In [21]:
df_clean.loc[df_clean['odeme_yontemi'].str.contains('kredi kartı', case=False), 'odeme_yontemi'] = 'Kredi Kartı'

In [22]:
df_clean["kategori"].value_counts()

kategori
Kozmetik      820
Spor          817
Ev & Yaşam    816
Elektronik    809
Giyim         803
Kitap         787
giyim          29
ev & yaşam     28
kozmetik       25
elektronik     24
spor           21
kitap          21
Name: count, dtype: int64

In [23]:
df_clean['kategori'] = (
    df_clean['kategori']
    .str.replace(r'\s+', ' ', regex=True)  # birden fazla boşluğu tek boşluk yap
    .str.strip()                            # baş ve sondaki boşlukları temizle
)

In [24]:
# for key, value in mapping.items():
   # df_clean.loc[df_clean['odeme_yontemi'].str.contains(key, case=False, na=False), 'odeme_yontemi'] = value

In [25]:
df_clean['kategori'] = df_clean['kategori'].str.upper()

In [26]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   siparis_id     5000 non-null   int64         
 1   tarih          4856 non-null   datetime64[ns]
 2   kategori       5000 non-null   object        
 3   fiyat          5000 non-null   float64       
 4   yas            4782 non-null   float64       
 5   sehir          5000 non-null   object        
 6   odeme_yontemi  5000 non-null   object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 273.6+ KB


In [27]:
df_clean = df_clean.drop("siparis_id", axis = 1)

In [28]:
df_clean.describe().T

,count,mean,min,25%,50%,75%,max,std
tarih,4856,2023-07-02 00:32:54.958813952,2023-01-01 00:00:00,2023-04-02 00:00:00,2023-07-02 00:00:00,2023-10-02 00:00:00,2024-01-01 00:00:00,NaN
fiyat,5000.0,2500.241992,-100.0,1261.42,2503.205,3737.2,4999.88,1440.849062
yas,4782.0,35.425136,-5.0,28.0,35.0,41.0,150.0,15.881283


In [29]:
# 0 TL altındaki fiyatları filtrele
alt_sifir = df_clean[df_clean["fiyat"] < 0]
len(alt_sifir)
# print(alt_sifir)

46

In [30]:
df_clean = df_clean[df_clean["fiyat"] >= 0]

In [31]:
df_clean.loc[(df_clean['yas'] < 0) | (df_clean['yas'] > 100), 'yas'] = np.nan